In [ ]:
"""
This script mapped the fastq files to the genome using cellranger, the script is called by the bash file 'run_count.sh'
author: Zhiyuan Yao

Update Date: Aug-10-2022
"""
import numpy as np
import pandas as pd
from copy import copy
import os
import glob
import shutil
import subprocess

In [ ]:
def grab_sample_list(input_path, sample_table = None):
    """Get the sample list which is ready for mapping, if the sample_table is provided, only mapped the sample on the table,
    otherwise scan the input folder and generate the sample list
    
    Parameters:
    ----------
    input_path: str
        The root path where the input folders to be created 
    sample_table: dataframe
        The table including the specified samples, the tables are from the tsv files generated by batch splitting process. 

    Returns:
        sample_list: list
        The sample list to be mapped
    """ 
    if sample_table is None:
        print("gathering sample lists from the input folder...")
        path_list = glob.glob(os.path.join(input_path,'*'))
        if len(path_list) == 0:
            raise IOError('No input subfolders')
        sample_list = [x.split('/')[-1] for x in path_list]
            
    else:
        print("gathering sample lists from the cumstom sample table...")
        sample_list = list(sample_table.name)
        
    print("{} samples are found:".format(len(sample_list)))
    for sample in sample_list:
        print(sample)
        
        
    return(sample_list)

def scan_input_folder(sample_list, input_path):
    """Scan the fastq files from the specified samples, check whether they have correct number of R1 and R2 files
    
    Parameters:
    ----------
    sample_list: list
        The sample list to be mapped
 
    input_path: str
        The root path where the input folders to be created 

    Returns:
        None
    """
    for sample in sample_list:
        fn_r1 = glob.glob(os.path.join(input_path,sample,'*R1_001.fastq.gz'))
        fn_r2 = glob.glob(os.path.join(input_path,sample,'*R2_001.fastq.gz'))
        
        if (len(fn_r1)!=len(fn_r2)):
            raise IOError('Sample {} does not have equal R1 and R2 fastqs'.format(sample))
        elif ((len(fn_r1) == 0) & (len(fn_r2) == 0)):
            raise IOError('Sample {} has no correctly re-named fastqs'.format(sample))
        else:
            print('Sample {} has correct fastqs'.format(sample))
            
    print('scan input folder completed, {} samples are ready to map'.format(len(sample_list)))
            
            

def make_output_folder(output_path, sample_list):
    """Generate output folders for the mapping results
    
    Parameters:
    ----------
    output_path: str
        The root path where the ouput folders to be created     
    sample_list: list
        The sample list to be mapped

    Returns:
        None
    """
    print("make output folders...")
    for sample in sample_list:
        print("creating the folder for sample {}".format(sample))
        folder_dir = os.path.join(output_path, sample)
        if not os.path.isdir(folder_dir):
            os.mkdir(folder_dir)
        else:
            raise Exception("The folder already exists")
    print("{} output folders has been created".format(len(sample_list)))
    
def cellranger_count(sample_list, input_path, output_path, cores = 40, mem = 64):  
    """Run cell ranger count
    
    Parameters:
    ----------
    sample_list: list
        The sample list to be mapped    
    input_path: str
        The root path where the input folders to be created         
    output_path: str
        The root path where the ouput folders to be created 
    cores: int
        max numbers of cpus to be used
    mem: int
        max GB memory to be used

    Returns:
        None
    """
    os.chdir(output_path)
    c = 0
    f = []
    for sample in sample_list:
        print("start mapping {}".format(sample))
        fastqs_path = os.path.join(input_path, sample)
        
#         cellranger_command = ["cellranger",
#                               "count",
#                               "--id={}".format(sample),
#                               "--transcriptome=/domino/edv/id-td-virology/Aligner/cellranger/refdata-gex-GRCh38-2020-A",
#                               "--fastqs={}".format(fastqs_path),
#                               "--sample={}".format(sample),
#                               "--include-introns=false",
#                               "--localcores={}".format(cores),
#                               "--localmem={}".format(mem)]
        
        cellranger_command = "cellranger count \
--id={} \
--transcriptome=/domino/edv/id-td-virology/Aligner/cellranger/refdata-gex-GRCh38-2020-A \
--fastqs={} \
--sample={} \
--include-introns=false \
--localcores={} \
--localmem={}".format(sample, fastqs_path, sample, cores, mem)
                
        a = subprocess.run(cellranger_command, capture_output=True, check = False, text = True, shell = True)        
        if a.returncode == 0:
            print('sample {} sucessfully processed'.format(sample))
            c = c + 1
        else:
            print("sample {} processing failed".format(sample))
            f.append(sample)
            continue
    
    print("mapping process is completed")        
    print("{} samples are sucessfully processed, {} samples failed".format(c, len(f)))
    if len(f) != 0:
        print("The failed samples:")
        for f_sample in f:
            print(f)

In [ ]:
if __name__ == '__main__':
    #root path of the input folders
    input_path = '/domino/edv/id-td-virology/Public_dataset/2022_Gut/input/'
    #root path of the output folders
    output_path = '/domino/edv/id-td-virology/Public_dataset/2022_Gut/output/'
    #files contain sample information from one batch, can be skipped if sample_table == none
    batch_path = '/domino/edv/id-td-virology/Public_dataset/2022_Gut/batch_file/s_batch_5.tsv'
    sample_table = pd.read_csv(batch_path, sep = '\t', index_col = 0)
    
    #generate sample list
    sample_list = grab_sample_list(input_path, sample_table = sample_table)
    #check the fastq files in the input folder are in the correct numbers
    scan_input_folder(sample_list, input_path)
    #generate the output folders
    make_output_folder(output_path, sample_list)
    #run cellranger count
    cellranger_count(sample_list, input_path, output_path, cores = 24, mem = 128)